In [1]:
import pymongo
from pymongo import MongoClient, GEOSPHERE
import shapely.geometry
import pandas as pd
from pandas import DataFrame
from dotenv import load_dotenv
import os
import requests
import json
from functools import reduce
import operator
import geopandas as gpd
import cartoframes
from cartoframes.viz import Map, Layer, popup_element

### 1. Connecting with Foursquare Api.

In [2]:
load_dotenv()
tok1 = os.getenv("tok1")
tok2 = os.getenv("tok2")

In [3]:
office_location = {'type': 'Point', 'coordinates': [40.7592189, -73.9783534]}
print (office_location)

{'type': 'Point', 'coordinates': [40.7592189, -73.9783534]}


### 2. Defining basskeetball stadiums' query

In [4]:
url_query = 'https://api.foursquare.com/v2/venues/explore'
basketball_stadium = "4bf58dd8d48988d18b941735"

In [5]:
# By taking information from Foursquare's API, we are looking for basketball stadiums around our office in a radius of 1000m.

parameters= {"client_id" : tok1,
              "client_secret" : tok2,
              "v": "20180323",
              "ll": f"{office_location.get('coordinates')[0]},{office_location.get('coordinates')[1]}",
              "query":f"basket_stadium",
                "radius":1000}

In [6]:
resp = requests.get(url= url_query, params = parameters)
data = json.loads(resp.text)

In [7]:
decoding_data = data.get("response")
decoded = decoding_data.get("groups")[0]
basket_stadium = decoded.get("items")

In [8]:
map_ = ["venue","name"]
m_latitude = ["venue","location","lat"]
m_longitude = ["venue","location","lng"]

In [9]:
def getFromDict(diccionario,mapa):
    return reduce (operator.getitem,mapa,diccionario)

In [10]:
print(getFromDict(basket_stadium[0],map_))

Regal E-Walk 4DX & RPX


In [11]:
basket_stadium_nyc = []
for dic in basket_stadium:
    list1 = {}
    list1["name"] = getFromDict(dic,map_)
    list1["latitud"] = getFromDict(dic,m_latitude)
    list1["longitud"] = getFromDict(dic,m_longitude)
    basket_stadium_nyc.append(list1)

In [12]:
# Conversion on dataframe that are used on maps construction.
df_stadium = pd.DataFrame(basket_stadium_nyc)
df_stadium.head()

,name,latitud,longitud
0,Regal E-Walk 4DX & RPX,40.756960,-73.989286
1,Zaro's Bakery,40.752875,-73.976946
2,Boomer Esiason's Stadium Grill,40.757840,-73.987099
3,Stadium Grill At Bowlmor Lanes,40.757211,-73.987395


**4 basketball stadiums** are located in a radius of 1000m. **Up to 22 increasing** the radius to 10km.

In [13]:
### Creating the column criteria
# Adding a criteria column
gdf_stadiums = gpd.GeoDataFrame(df_stadium, geometry = gpd.points_from_xy(df_stadium.longitud,df_stadium.latitud))
gdf_stadiums = gdf_stadiums.assign(name = [ "Regal E-Walk 4DX & RPX",
                           "Zaro's Bakery",
                           "Boomer Esiason's Stadium Grill",
                           "Stadium Grill At Bowlmor Lanes"],
               criteria = ["basketball stadium","basketball stadium","basketball stadium","basketball stadium"])
gdf_stadiums.head()

,name,latitud,longitud,geometry,criteria
0,Regal E-Walk 4DX & RPX,40.756960,-73.989286,POINT (-73.98929 40.75696),basketball stadium
1,Zaro's Bakery,40.752875,-73.976946,POINT (-73.97695 40.75288),basketball stadium
2,Boomer Esiason's Stadium Grill,40.757840,-73.987099,POINT (-73.98710 40.75784),basketball stadium
3,Stadium Grill At Bowlmor Lanes,40.757211,-73.987395,POINT (-73.98739 40.75721),basketball stadium


In [14]:
# Convert our dataframe on a dict to be able to use it in MongoDB.

In [15]:
gdf_stadiums["geometry"] = gdf_stadiums["geometry"].apply(lambda x: shapely.geometry.mapping(x))
gdf_stadiums

/usr/local/lib/python3.9/site-packages/geopandas/geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


,name,latitud,longitud,geometry,criteria
0,Regal E-Walk 4DX & RPX,40.756960,-73.989286,"{'type': 'Point', 'coordinates': (-73.98928566...",basketball stadium
1,Zaro's Bakery,40.752875,-73.976946,"{'type': 'Point', 'coordinates': (-73.976946, ...",basketball stadium
2,Boomer Esiason's Stadium Grill,40.757840,-73.987099,"{'type': 'Point', 'coordinates': (-73.98709862...",basketball stadium
3,Stadium Grill At Bowlmor Lanes,40.757211,-73.987395,"{'type': 'Point', 'coordinates': (-73.98739450...",basketball stadium


In [16]:
type(gdf_stadiums.loc[0,"geometry"])

dict

### 3. Export the query to Mongo

In [17]:
client = MongoClient() 
db = client.companies
collection = db.basket_stadium 
collection.create_index ([("geometry", GEOSPHERE)])

'geometry_2dsphere'

In [18]:
data = gdf_stadiums.to_dict(orient='records')
data[0]

{'name': 'Regal E-Walk 4DX & RPX',
 'latitud': 40.756960133814175,
 'longitud': -73.9892856652269,
 'geometry': {'type': 'Point',
  'coordinates': (-73.9892856652269, 40.756960133814175)},
 'criteria': 'basketball stadium'}

In [19]:
collection.insert_many(data)

### 4. Determine the nearest basket stadium.

In [20]:
db.basket_stadium

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies'), 'basket_stadium')

In [21]:
filt = {"geometry":{"$near":{"type":"Point","coordinates":[40.7592189, -73.9783534]}}}
proyect = {"_id":0,"name":1,"latitud":1,"longitud":1}

result = db.basket_stadium.find(filt,proyect).limit(1)
basket_stadium = list(result)

In [22]:
basket_stadium

[{'name': "Zaro's Bakery", 'latitud': 40.752875, 'longitud': -73.976946}]